# Molcule generation benchmark challenge, CS 598 GBL, group 11

The FlowMol and GeoLDM models were used for sample generation in this experiment.

# FlowMol
The evnironemt set up process for this model was follwed as meniotned in the demo presented at the course repo. Then pretrained models were downloaded by running this script form the root of the [FlowMol repository](https://https://github.com/Dunni3/FlowMol)

In [ ]:
wget -r -np -nH --cut-dirs=3 --reject 'index.html*' -P flowmol/trained_models/ https://bits.csb.pitt.edu/files/FlowMol/trained_models_v02/

After downloading the pretrained model, the qm9 dataset was downloaded by running this line from the root of the repo:


In [ ]:
mkdir data/qm9_raw
cd data/qm9_raw
wget https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/molnet_publish/qm9.zip
wget -O uncharacterized.txt https://ndownloader.figshare.com/files/3195404
unzip qm9.zip

cmtc was the first configuration used as a baseline for this experiment. Therefore qm9 dataset got processed by running this script from the root of the repo

In [ ]:
python process_qm9.py --config=configs/qm9_ctmc.yaml

After data processing 10 k samples were generated by running this script. The samples will be saved in the model directory by default.

In [ ]:
python test.py --model_dir flowmol/trained_models/qm9_ctmc/ --n_mols 10000 --n_timesteps 100

And for the gaussian model:


In [ ]:
python process_qm9.py --config=configs/qm9_gaussian.yaml
python test.py --model_dir flowmol/trained_models/qm9_gaussian/ --n_mols 10000 --n_timesteps 100

The process was repeated for the geom-drug dataset as well. To download the data:

In [ ]:
wget -r -np -nH --cut-dirs=2 --reject "index.html*" -P data/ https://bits.csb.pitt.edu/files/geom_raw/

We should move the downloaded files to geom_raw folder. To process and sample using ctmc model:




In [ ]:
python process_geom.py data/geom_raw/train_data.pickle --config=configs/geom_ctmc.yaml
python process_geom.py data/geom_raw/test_data.pickle --config=configs/geom_ctmc.yaml
python process_geom.py data/geom_raw/val_data.pickle --config=configs/geom_ctmc.yaml
python test.py --model_dir flowmol/trained_models/qm9_ctmc/ --n_mols 10000 --n_timesteps 250

To process and sample using gaussian model:

In [ ]:
python process_geom.py data/geom_raw/train_data.pickle --config=configs/geom_gaussian.yaml
python process_geom.py data/geom_raw/test_data.pickle --config=configs/geom_gaussina.yaml
python process_geom.py data/geom_raw/val_data.pickle --config=configs/geom_gaussian.yaml
python test.py --model_dir flowmol/trained_models/geom_gaussian/ --n_mols 10000 --n_timesteps 250

The saved sampled then were evaluated against instance level metrics by running these cells. The path to the samples of the desired model should get uncommented each time.
To load samples:

In [ ]:
from rdkit import Chem, RDLogger
RDLogger.DisableLog('rdApp.*')
import pickle as pkl
import torch
from tqdm import tqdm

# change to your path

path = 'flowmol/trained_models/qm9_ctmc/samples/sampled_mols.sdf'
#path = 'flowmol/trained_models/qm9_gaussian/samples/sampled_mols.sdf'
#path = 'flowmol/trained_models/geom_ctmc/samples/sampled_mols.sdf'
#path = 'flowmol/trained_models/geom_gaussian/samples/sampled_mols.sdf'

supplier = Chem.SDMolSupplier(path)

mols = [mol for mol in supplier]
print(f'{len(mols)} mols loaded')

To visualize one of the generated molcules:

In [ ]:
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
import py3Dmol
from IPython.display import display

# show 2d mol
mol = mols[0]
img = Draw.MolToImage(mol, size=(300, 300))
display(img)

# show 3d mol
mb = Chem.MolToMolBlock(mol)
viewer = py3Dmol.view(width=400, height=400)
viewer.addModel(mb, 'mol')
viewer.setStyle({'stick': {}})
viewer.zoomTo()
viewer.show()

To evaluate the average molcule size, atom and molcule stability, number of valid and complete molcules, averagr MMFF energy and uniqueness:

In [ ]:
from tqdm import tqdm
from collections import Counter
import numpy as np
from rdkit.Chem import AllChem
from copy import deepcopy

# https://github.com/Dunni3/FlowMol/issues/12
allowed_bonds = {'H': {0: 1, 1: 0},
                 'C': {0: 4},
                 'N': {0: 3, 1: 4, -1: 2},
                 'O': {0: 2, 1: 3, -1: 1},
                 'F': {0: 1, -1: 0},
                 'B': 3, 'Al': 3, 'Si': 4,
                 'P': {0: [3, 5], 1: 4},
                 'S': {0: [2, 6], 1: [2, 3], 2: 4, 3: 5, -1: 3},
                 'Cl': 1, 'As': 3,
                 'Br': {0: 1, 1: 2}, 'I': 1, 'Hg': [1, 2], 'Bi': [3, 5], 'Se': [2, 4, 6]}
bond_dict = [None, Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE, Chem.rdchem.BondType.TRIPLE,
             Chem.rdchem.BondType.AROMATIC]

def check_stability(mol):
    atom_types = [atom.GetSymbol() for atom in mol.GetAtoms()]
    valencies = [atom.GetTotalValence() for atom in mol.GetAtoms()]
    charges = [atom.GetFormalCharge() for atom in mol.GetAtoms()]

    n_stable_atoms = 0
    mol_stable = True
    for i, (atom_type, valency, charge) in enumerate(zip(atom_types, valencies, charges)):
        valency = int(valency)
        charge = int(charge)
        possible_bonds = allowed_bonds[atom_type]
        if type(possible_bonds) == int:
            is_stable = possible_bonds == valency
        elif type(possible_bonds) == dict:
            expected_bonds = possible_bonds[charge] if charge in possible_bonds.keys() else possible_bonds[0]
            is_stable = expected_bonds == valency if type(expected_bonds) == int else valency in expected_bonds
        else:
            is_stable = valency in possible_bonds
        if not is_stable:
            mol_stable = False
        n_stable_atoms += int(is_stable)

    return n_stable_atoms, mol_stable, atom_types


def check_validity(mol):
    valid = False
    complete = False

    if mol is None:
        return False

    mol_frags = Chem.rdmolops.GetMolFrags(mol, asMols=True, sanitizeFrags=False)
    n_frags = len(mol_frags)

    if n_frags == 1:
        complete = True

    try:
        Chem.SanitizeMol(mol)
        smiles = Chem.MolToSmiles(mol)
        valid = True
    except Error as e:
        print(e)

    return valid and complete


def get_molecule_force_field(mol, conf_id=None, force_field='mmff', **kwargs):
    """
    Get a force field for a molecule.
    Parameters
    ----------
    mol : RDKit Mol
        Molecule.
    conf_id : int, optional
        ID of the conformer to associate with the force field.
    force_field : str, optional
        Force Field name.
    kwargs : dict, optional
        Keyword arguments for force field constructor.
    """
    if force_field == 'uff':
        ff = AllChem.UFFGetMoleculeForceField(
            mol, confId=conf_id, **kwargs)
    elif force_field.startswith('mmff'):
        AllChem.MMFFSanitizeMolecule(mol)
        mmff_props = AllChem.MMFFGetMoleculeProperties(
            mol, mmffVariant=force_field)
        ff = AllChem.MMFFGetMoleculeForceField(
            mol, mmff_props, confId=conf_id, **kwargs)
    else:
        raise ValueError("Invalid force_field {}".format(force_field))
    return ff


def get_conformer_energies(mol, force_field='mmff'):
    """
    Calculate conformer energies.
    Parameters
    ----------
    mol : RDKit Mol
        Molecule.
    force_field : str, optional
        Force Field name.
    Returns
    -------
    energies : array_like
        Minimized conformer energies.
    """
    energies = []
    for conf in mol.GetConformers():
        ff = get_molecule_force_field(mol, conf_id=conf.GetId(), force_field=force_field)
        energy = ff.CalcEnergy()
        energies.append(energy)
    energies = np.asarray(energies, dtype=float)
    return energies


def evaluate(mols):
    n_atoms = 0
    n_stable_atoms = 0
    n_stable_mols = 0
    n_mols = 0

    n_valid_mols = 0
    n_complete_mols = 0
    n_valid_and_complete_mols = 0

    all_atom_types = []
    valid_mols = []
    all_energy = []
    all_smiles = []

    for mol in tqdm(mols, ncols=100):
        if mol is None:
            continue

        n_mols += 1

        n_atoms += mol.GetNumAtoms()
        _n_stable_atoms, _mol_stable, _atom_types = check_stability(mol)
        n_stable_atoms += _n_stable_atoms
        n_stable_mols += int(_mol_stable)
        all_atom_types.extend(_atom_types)

        try:
            Chem.SanitizeMol(mol)
            smiles = Chem.MolToSmiles(mol)
            _mol_valid = True

            mol_frags = Chem.rdmolops.GetMolFrags(mol, asMols=True, sanitizeFrags=False)
            n_frags = len(mol_frags)
            _mol_complete = n_frags == 1

            n_valid_mols += int(_mol_valid)
            n_complete_mols += int(_mol_complete)
            n_valid_and_complete_mols += int(_mol_valid and _mol_complete)

            if not (_mol_valid and _mol_complete):
                continue

            valid_mols.append(mol)
            all_smiles.append(smiles)

            energy = get_conformer_energies(mol)
            all_energy.append(energy)
        except Exception as e:
            print(e)

    frac_atom_stable = n_stable_atoms / n_atoms
    frac_mol_stable = n_stable_mols / n_mols
    frac_mol_valid = n_valid_mols / n_mols
    frac_mol_complete = n_complete_mols / n_mols
    frac_mol_valid_and_complete = n_valid_and_complete_mols / n_mols

    print(f'avg mol size: {n_atoms / n_mols:.2f}')
    print(f'atom stable: {frac_atom_stable:.2f}, mol stable: {frac_mol_stable:.2f}')
    print(f'valid: {frac_mol_valid:.2f}, complete: {frac_mol_complete:.2f}, valid and complete: {n_valid_and_complete_mols:.2f}')

    all_energy = np.array(all_energy)
    all_energy = all_energy[~np.isnan(all_energy)]
    mean_energy = np.mean(all_energy)

    print(f'mean mmff energy: {mean_energy:.2f} from {len(all_energy)} mols')

    unique = len(set(all_smiles)) / len(all_smiles)
    print(f'unique: {unique:.2f}')

    return valid_mols, all_smiles, all_energy, all_atom_types

valid_mols, all_smiles, all_energy, all_atom_types = evaluate(mols)
print(f'{len(valid_mols)} valid mols')

The calculation of distribution level metrics such as novelty and diversity was this size of samples was computationaly heavy and time consuming and our team did not have proper resources to calculate them using the script suggested by the demo notebook.

# GeoLDM
After cloning to https://github.com/MinkaiXu/GeoLDM, and downloading data and pretrained models.The environment and requiments were set up following the instrcution in GeoLDM repository by running these scripts in the terminal. The simple_req.txt includes the simpler version of requirments needed for the model.This file can be found in [our repo](https://github.com/RJahedan/CS598GBLGroup11). In addition to the packages listed in the repository, openbable should also get installed to suport the conformer energy calcuation process which is added to the evaluation part by our team.

In [ ]:
conda create -c conda-forge -n my-rdkit-env rdkit
conda activate my-redkit-env
pip install -r simple_req.text
pip install openbabel-wheel


To generate and anlyze the samples, My_Eval.py and reconstruct.py should get added to the GeoLDM repo root. The original  python script which was used for sampling and evaluation was edited by our group to add save and load option for samples as well as adding the average molcule size and conformer energy to the evaloation metrics. To generate and analyze samples for qm9 dataset:

In [ ]:
python My_Eval.py --model_path outputs/$qm9 --n_samples 10000 --save_samples_to GeoLDM_qm9.pickle

To load previuosly saved samples and analyze them:


In [ ]:
python My_Eval.py --model_path outputs/$qm9 --n_samples 10000 --load_samples GeoLDM_qm9.pickle

For geom_drug dataset:

In [ ]:
python My_Eval.py --model_path outputs/$geom --n_samples 10000 --save_samples_to geoldm_drug.pickle